## Import Libraries

In [12]:
import tensorflow as tf #Deep learning library
import numpy as np #Matrix algebra library
from tensorflow.examples.tutorials.mnist import input_data #Import training data
import pandas as pd #Database management library
import matplotlib.pyplot as plt #Visualization library 
%matplotlib inline  
from sklearn.svm import LinearSVC #For linear image classification
from sklearn.model_selection import GridSearchCV #For hyperparameter optimization

## Load MNIST Dataset & Prepare Data for RBM 

We will be building this RBM in Tensorflow and testing it on the MNIST dataset. We will not need one_hot encoding as we will be using our RBM to extract important features from our image data to be used in a linear classifier. 

In [13]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False) 
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Restricted Boltzmann Machine

### Who is this explanation for?  

At the risk of sacrificing some important information, I will be discussing this with you as I would a close friend who knew a little about deep learning. Sometimes, I think its nice to get a general overview of how something works before diving into the specifics. If you are someone like me who was taking Geoffrey Hinton's Coursera course and had no idea what he was talking about when he got into Restricted Boltzmann Machines...keep reading! 

### How will we be utilizing this RBM?

When my research team began developing a new method of feature engineering for image classification, they were very interested to see how it compared to the classic RBM. Naturally, I was given the tedious task of building one and producing benchmarks for a variety of datasets. The goal of this tutorial is to utilize the RBM to extract important features from an image, and then use those features in a simple linear classifier. We want to take image data that is NOT linearly seperable and attempt to make it... more linearly seperable by using the RBM as a feature engineering tool. This is only one of many use cases for the RBM.


### How do we do that? 

The RBM is an unsupervised model, meaning, there is no known output we are mapping our data to. We have our input layer (which is just the pixels of our image) and a hidden layer which will, over time, be learning what the most important features are. Now I know I said I am assuming you know a little about deep learning, but before we get into how RBMs work we should probably set a few definitions. 

Back propagation: This is a tool that updates the weights of our neural network in a direction that minimizes our error. Over time, these weights will allow our neural network (or RBM in this case) to learn whatever it is we are asking it to learn. 

Dense layer: In our case, this simply means that every neuron in the input layer is connected to every neuron of the hidden layer. This does not mean that neurons in the input layer are connected to other neurons in the input layer. Input layer neurons are only connected to neurons in the hidden layer. 

Input Layer: The layer of the RBM that contains the original image data. 

Hidden Layer: The layer of the RBM that is extracting the key features from the image. The goal of the hidden layer is to eventually reconstruct what it sees in the input layer without using all of the original pixels. It only uses features it deems most important. Lets talk about how this happens.

### Inside of the RBM

So this is what's going down. Lets take one image from the MNIST dataset and discuss what happens as it goes through a restricted boltzmann machine. MNIST is full of 28x28 images of hand-written digits. First, we take our 28x28 image and flatten it to a 1D array of length 784 (because 28*28 = 784). Now, if you are picturing an artificial neural network in your head, these are all of the neurons of the input layer. Now we need to decide how many neurons we wish to have in the hidden layer. 

The hidden layer is a magical place. A place where you can reconstruct the original image with less neurons than you started with. What?!??!? Thats right! The goal of the hidden layer is to extract the most important features from the original image and then use those features to reconstruct the original image! So earlier I asked the question: How many neurons should we have in the hidden layer? The answer: I dont know. As you may know by now, machine learning is an iterative process and finding the magic number will take some time. Often, people will reduce the number of neurons in the hidden layer, for example, going from 784 neurons to 650 neurons. Thus, the RBM now has to work hard to capture the essence of the original image in only 650 neurons. How does it do this? Well, lets stop and recap what we are currently looking at. 

1) Image gets flattened into a 784 length array and prepared for the input layer of the RBM. 

2) We decided that the hidden layer size is 650. So now we will randomly initalize our weights that are connecting each neuron from the input layer to each neuron of the hidden layer. 

These weights connecting the two layers are what is going to allow the RBM to model the original image with a smaller amount of neurons. Using back propagation, the weights will, over time, learn how to help the hidden layer represent the original image with a smaller number of pixels. 

### How do we know if it's working? 

So we have inputted our image, created our hidden layer, and our weights are randomly initalized and ready to go. Now what? Well, the RBM is going to attempt to reconstruct the original 28*28 image with only 650 neurons. On its first go around, it will probably do pretty bad considering we gave the model random weights to work with. We can check how bad it did by measuring the difference between the hidden layer reconstruction and the original image. After that, the RBM will go through another iteration, this time updating the weights to help the hidden layer reconstruction better represent the original image, and again, we check to see the difference between the original image and the reconstruction. This proceess goes on and on until we find that our error is low and the reconstructed image closely resembles the original image.

### Linear classification time! 

Now that our hidden layer is able to reconstruct the original image extremely well with only 650 neurons, lets see how these 650 neurons perform in a linear classifier. With no feature engineering, meaning, just using our 784 original neurons as our training data, I was able to get 91% accuracy using a LinearSVC. After running the RBM, extracting the 650 neurons containing our high quality features from the hidden layer, and using those features as training data, I got 97% accuracy! I could probably get better results if I experimented with different numbers of neurons in my hidden layer. Let me know if you beat 97%!

### Now its your turn!

Follow along with the code below. I have heavily commented it for you and if you have any questions feel free to email me. 

### Contrastive Divergence

If you are going through Geoffrey Hinton's course and learned about Contrastive Divergence, I have a tensorflow implementation for you. Feel free to try it out. I personally have never come across a time where CD-n for n > 1 performed better than n = 1 but after hearing so much about it I had to try. 

For those wondering, all Contrastive Divergence means is, instead of the RBM iterating over and over again going from original input to reconstruction > original input to better reconstruction and so on... now you are using the reconstruction as the input layer in the second iteration... so original input to reconstruction to reconstruction of the reconstruction and so on. It's not very popular but can be fun to play with! 



In [22]:
class RBM(object):
    def __init__(self, input_size, output_size, learning_rate, batch_size):
        self.input_size = input_size #Size of the input layer
        self.output_size = output_size #Size of the hidden layer
        self.epochs = 2 #How many times we will update the weights 
        self.learning_rate = learning_rate #How big of a weight update we will perform 
        self.batch_size = batch_size #How many images will we "feature engineer" at at time 
        self.new_input_layer = None #Initalize new input layer variable for k-step contrastive divergence 
        self.new_hidden_layer = None
        self.new_test_hidden_layer = None
        
        #Here we initialize the weights and biases of our RBM
        #If you are wondering, the 0 is the mean of the distribution we are getting our random weights from. 
        #The .01 is the standard deviation.
        self.w = np.random.normal(0,.01,[input_size,output_size]) #weights
        self.hb = np.random.normal(0,.01,[output_size]) #hidden layer bias
        self.vb = np.random.normal(0,.01,[input_size]) #input layer bias (sometimes called visible layer)
        
        
        #Calculates the sigmoid probabilities of input * weights + bias
        #Here we multiply the input layer by the weights and add the bias
        #This is the phase that creates the hidden layer
    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)
        
        #Calculates the sigmoid probabilities of input * weights + bias
        #Here we multiply the hidden layer by the weights and add the input layer bias
        #This is the reconstruction phase that recreates the original image from the hidden layer
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    #Returns new layer binary values
    #This function returns a 0 or 1 based on the sign of the probabilities passed to it
    #Our RBM will be utilizing binary features to represent the images
    #This function just converts the features we have learned into a binary representation 
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))
    
    def train(self, X, teX):
        #Initalize placeholder values for graph
        #If this looks strange to you, then you have not used Tensorflow before
        _w = tf.placeholder(tf.float32, shape = [self.input_size, self.output_size])
        _vb = tf.placeholder(tf.float32, shape = [self.input_size])
        _hb = tf.placeholder(tf.float32, shape = [self.output_size])
        
        
        #initalize previous variables
        #we will be saving the weights of the previous and current iterations
        pre_w = np.random.normal(0,.01, size = [self.input_size,self.output_size])
        pre_vb = np.random.normal(0, .01, size = [self.input_size])
        pre_hb = np.random.normal(0, .01, size = [self.output_size])
        
        #initalize current variables
        #we will be saving the weights of the previous and current iterations
        cur_w = np.random.normal(0, .01, size = [self.input_size,self.output_size])
        cur_vb = np.random.normal(0, .01, size = [self.input_size])
        cur_hb = np.random.normal(0, .01, size = [self.output_size])
               
        #Plaecholder variable for input layer
        v0 = tf.placeholder(tf.float32, shape = [None, self.input_size])
        
        #pass probabilities of input * w + b into sample prob to get binary values of hidden layer
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb ))
        
        #pass probabilities of new hidden unit * w + b into sample prob to get new reconstruction
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        
        #Just get the probailities of the next hidden layer. We wont need the binary values. 
        #The probabilities here help calculate the gradients during back prop 
        h1 = self.prob_h_given_v(v1, _w, _hb)
        
        
        #Contrastive Divergence
        positive_grad = tf.matmul(tf.transpose(v0), h0) #input' * hidden0
        negative_grad = tf.matmul(tf.transpose(v1), h1) #reconstruction' * hidden1
        #(pos_grad - neg_grad) / total number of input samples 
        CD = (positive_grad - negative_grad) / tf.to_float(tf.shape(v0)[0]) 
        
        #This is just the definition of contrastive divergence 
        update_w = _w + self.learning_rate * CD
        update_vb = _vb + tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb + tf.reduce_mean(h0 - h1, 0)
        
        #MSE - This is our error function
        err = tf.reduce_mean(tf.square(v0 - v1))
        
        #Will hold new visible layer.
        errors = []
        hidden_units = []
        reconstruction = []
        
        test_hidden_units = []
        test_reconstruction=[]
        
        
        #The next four lines of code intitalize our Tensorflow graph and create mini batches
        #The mini batch code is from cognitive class. I love the way they did this. Just giving credit! 
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):
                for start, end in zip(range(0, len(X), self.batch_size), range(self.batch_size, len(X), self.batch_size)):
                    batch = X[start:end] #Mini batch of images taken from training data
                    
                    #Feed in batch, previous weights/bias, update weights and store them in current weights
                    cur_w = sess.run(update_w, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    cur_hb = sess.run(update_hb, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    cur_vb = sess.run(update_vb, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    
                    #Save weights 
                    pre_w = cur_w
                    pre_hb = cur_hb
                    pre_vb = cur_vb
                
                #At the end of each iteration, the reconstructed images are stored and the error is outputted 
                reconstruction.append(sess.run(v1, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb}))        
                print('Learning Rate: {}:  Batch Size: {}:  Hidden Layers: {}: Epoch: {}: Error: {}:'.format(self.learning_rate, self.batch_size, 
                                                                                                             self.output_size, (epoch+1),
                                                                                                            sess.run(err, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb})))
            
            #Store final reconstruction in RBM object
            self.new_input_layer = reconstruction[-1]
            
            #Store weights in RBM object
            self.w = pre_w
            self.hb = pre_hb
            self.vb = pre_vb
    
    #This is used for Contrastive Divergence.
    #This function makes the reconstruction your new input layer. 
    def rbm_output(self, X):
        input_x = tf.constant(X)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        _vb = tf.constant(self.vb)
        
        out = tf.nn.sigmoid(tf.matmul(input_x, _w) + _hb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out)
            

In [23]:
#A function for training your RBM
#Keep k at 1 for a traditional RBM. 
def train_cd_k(k, input_size, output_size, trX, teX, learning_rate, batch_size):
    cdk_train_input = trX #Training data
    cdk_test_input = teX  #Testing data
    cdk_train_hidden = None #Variable to store hidden layer for training data
    cdk_test_hidden = None  #Variable to store hidden layer for testing data
    
    rbm = RBM(input_size, output_size, learning_rate, batch_size)
    
    #Loop for contrastive divergence. 
    for i in range(k):
        print('CD: {}'.format(int(i+1)))
        rbm.train(cdk_train_input, cdk_test_input) #Using reconstruction as input layer for CD
        cdk_train_input = rbm.new_input_layer
        cdk_train_hidden = rbm.rbm_output(cdk_train_input)
        cdk_test_hidden = rbm.rbm_output(cdk_test_input)
        
    return [cdk_train_hidden, cdk_test_hidden, cdk_train_input]
    

In [ ]:
with tf.Graph().as_default():
    temp = train_cd_k(1,784,650,trX,teX,.001,32)
    lsvc_RBM = LinearSVC()
    lsvc_RBM.fit(temp[cdk_train_hidden], trY)
    lsvc_RBM.score(temp[cdk_test_hidden], teY)